In [1]:
!apt-get -y install libenchant-dev

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following package was automatically installed and is no longer required:
  libnvidia-common-440
Use 'apt autoremove' to remove it.
The following additional packages will be installed:
  aspell aspell-en dictionaries-common emacsen-common enchant hunspell-en-us
  libaspell15 libenchant1c2a libhunspell-1.6-0 libtext-iconv-perl
Suggested packages:
  aspell-doc spellutils wordlist hunspell openoffice.org-hunspell
  | openoffice.org-core libenchant-voikko
The following NEW packages will be installed:
  aspell aspell-en dictionaries-common emacsen-common enchant hunspell-en-us
  libaspell15 libenchant-dev libenchant1c2a libhunspell-1.6-0
  libtext-iconv-perl
0 upgraded, 11 newly installed, 0 to remove and 35 not upgraded.
Need to get 1,362 kB of archives.
After this operation, 5,602 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu bionic/main amd64 libtext-iconv

In [2]:
!pip install pyenchant

     |████████████████████████████████| 61kB 1.8MB/s 


In [3]:
import re
import enchant
import nltk

from nltk.corpus import stopwords, words
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.tokenize import word_tokenize


nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


True

In [4]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

In [5]:
from google.colab import files
data_to_load = files.upload()

Saving formspring_data.csv to formspring_data.csv


In [6]:
df = pd.read_csv('formspring_data.csv', sep='\t')

In [7]:
df

,userid,post,ques,ans,asker,ans1,severity1,bully1,ans2,severity2,bully2,ans3,severity3,bully3
0,aguitarplayer94,Q: what&#039;s your favorite song? :D<br>A: I ...,what&#039;s your favorite song? :D<br>,I like too many songs to have a favorite,None,No,0,NaN,No,0,NaN,No,0,NaN
1,aprilpooh15,Q: <3<br>A: </3 ? haha jk! <33,<3,</3 ? haha jk! <33,None,No,0,NaN,No,0,NaN,No,0,NaN
2,aprilpooh15,Q: &quot;hey angel you duh sexy&quot;<br>A: R...,&quot;hey angel you duh sexy&quot;,Really?!?! Thanks?! haha,None,No,0,NaN,No,0,NaN,No,0,NaN
3,aprilpooh15,Q: (:<br>A: ;(,(:,;(,None,No,0,NaN,No,0,NaN,No,0,NaN
4,aprilpooh15,Q: ******************MEOWWW*******************...,******************MEOWWW*************************,*RAWR*?,None,No,0,NaN,No,0,NaN,No,0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12768,outlaw9000,Q: Youre at a party and your friend who drove ...,Youre at a party and your friend who drove you...,take the battery cable off and make her sta...,None,No,0,NaN,No,0,NaN,No,0,NaN
12769,outlaw9000,Q: Youu2019re awesome :] Give yourself a compl...,Youu2019re awesome :] Give yourself a complime...,thank you ! ! I AM Awesome ! !,None,No,0,None,No,0,NaN,No,0,NaN
12770,outlaw9000,Q: Yu play with yurself ?<br>A: all the time. ...,Yu play with yurself ?,all the time. sometimes more than once a day,None,No,0,NaN,No,0,NaN,No,0,NaN
12771,outlaw9000,Q: yukk beer is disgusting. i can only drink i...,yukk beer is disgusting. i can only drink it w...,oh O.K ! ! nice make sure it's Stolie's Please,None,No,0,NaN,No,0,NaN,No,0,NaN


In [11]:
df.columns

Index(['userid', 'post', 'ques', 'ans', 'asker', 'ans1', 'severity1', 'bully1',
       'ans2', 'severity2', 'bully2', 'ans3', 'severity3', 'bully3'],
      dtype='object')

In [12]:
df.describe()

,userid,post,ques,ans,asker,ans1,severity1,bully1,ans2,severity2,bully2,ans3,severity3,bully3
count,12773,12773,12771,12770,12773,12731,12772,1909,12735,12773,1806,12728,12773,1826
unique,49,12771,12326,11993,552,2,12,1013,2,15,1002,2,15,982
top,zooshay,Q: Wait. Are you Cheyenne? I sent a request to...,hi,nope,None,No,0,None,No,0,None,No,0,None
freq,2245,2,7,50,6790,11693,11484,593,11710,11526,527,11740,11514,551


**Three Results**
 The data has three results, and I would like to only view data where all the three results match.

In [13]:
sum(df.ans1 == df.ans2) / len(df)


0.9087919830893291

In [14]:
sum(df.ans2 == df.ans3) / len(df)

0.9145071635481093

In [15]:
sum(df.ans1 == df.ans3) / len(df)

0.9112189775307289

In [39]:
index = (df.ans1 == df.ans2) & (df.ans2 == df.ans3) & (df.ans1 == df.ans3)  
 ##A post was considered harmful if atleast two out of three annotators answered “yes” for the primary question.

In [17]:
df = df[index]

In [18]:
df

,userid,post,ques,ans,asker,ans1,severity1,bully1,ans2,severity2,bully2,ans3,severity3,bully3
0,aguitarplayer94,Q: what&#039;s your favorite song? :D<br>A: I ...,what&#039;s your favorite song? :D<br>,I like too many songs to have a favorite,None,No,0,NaN,No,0,NaN,No,0,NaN
1,aprilpooh15,Q: <3<br>A: </3 ? haha jk! <33,<3,</3 ? haha jk! <33,None,No,0,NaN,No,0,NaN,No,0,NaN
2,aprilpooh15,Q: &quot;hey angel you duh sexy&quot;<br>A: R...,&quot;hey angel you duh sexy&quot;,Really?!?! Thanks?! haha,None,No,0,NaN,No,0,NaN,No,0,NaN
3,aprilpooh15,Q: (:<br>A: ;(,(:,;(,None,No,0,NaN,No,0,NaN,No,0,NaN
4,aprilpooh15,Q: ******************MEOWWW*******************...,******************MEOWWW*************************,*RAWR*?,None,No,0,NaN,No,0,NaN,No,0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12766,outlaw9000,Q: Your parents are divorced. One of them gets...,Your parents are divorced. One of them gets ma...,Not incest because we would not be related s...,None,No,0,NaN,No,0,NaN,No,0,NaN
12768,outlaw9000,Q: Youre at a party and your friend who drove ...,Youre at a party and your friend who drove you...,take the battery cable off and make her sta...,None,No,0,NaN,No,0,NaN,No,0,NaN
12769,outlaw9000,Q: Youu2019re awesome :] Give yourself a compl...,Youu2019re awesome :] Give yourself a complime...,thank you ! ! I AM Awesome ! !,None,No,0,None,No,0,NaN,No,0,NaN
12770,outlaw9000,Q: Yu play with yurself ?<br>A: all the time. ...,Yu play with yurself ?,all the time. sometimes more than once a day,None,No,0,NaN,No,0,NaN,No,0,NaN


In [19]:
# Putting back apostrophe's may help
df.ques = df.ques.str.replace("&#039;", "'") 

# Removing may not be effective since there is a word check implemented
df.ques = df.ques.str.replace("<br>", "") 
df.ques = df.ques.str.replace("&quot;", "") 
df.ques = df.ques.str.replace("<3", "") 

/usr/local/lib/python3.6/dist-packages/pandas/core/generic.py:5303: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[name] = value


In [20]:
df

,userid,post,ques,ans,asker,ans1,severity1,bully1,ans2,severity2,bully2,ans3,severity3,bully3
0,aguitarplayer94,Q: what&#039;s your favorite song? :D<br>A: I ...,what's your favorite song? :D,I like too many songs to have a favorite,None,No,0,NaN,No,0,NaN,No,0,NaN
1,aprilpooh15,Q: <3<br>A: </3 ? haha jk! <33,,</3 ? haha jk! <33,None,No,0,NaN,No,0,NaN,No,0,NaN
2,aprilpooh15,Q: &quot;hey angel you duh sexy&quot;<br>A: R...,hey angel you duh sexy,Really?!?! Thanks?! haha,None,No,0,NaN,No,0,NaN,No,0,NaN
3,aprilpooh15,Q: (:<br>A: ;(,(:,;(,None,No,0,NaN,No,0,NaN,No,0,NaN
4,aprilpooh15,Q: ******************MEOWWW*******************...,******************MEOWWW*************************,*RAWR*?,None,No,0,NaN,No,0,NaN,No,0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12766,outlaw9000,Q: Your parents are divorced. One of them gets...,Your parents are divorced. One of them gets ma...,Not incest because we would not be related s...,None,No,0,NaN,No,0,NaN,No,0,NaN
12768,outlaw9000,Q: Youre at a party and your friend who drove ...,Youre at a party and your friend who drove you...,take the battery cable off and make her sta...,None,No,0,NaN,No,0,NaN,No,0,NaN
12769,outlaw9000,Q: Youu2019re awesome :] Give yourself a compl...,Youu2019re awesome :] Give yourself a complime...,thank you ! ! I AM Awesome ! !,None,No,0,None,No,0,NaN,No,0,NaN
12770,outlaw9000,Q: Yu play with yurself ?<br>A: all the time. ...,Yu play with yurself ?,all the time. sometimes more than once a day,None,No,0,NaN,No,0,NaN,No,0,NaN


In [22]:
bully_df = df[df.ans1 == "Yes"].reset_index(drop=True)
df[df.ans1 == "Yes"].head()                                      

,userid,post,ques,ans,asker,ans1,severity1,bully1,ans2,severity2,bully2,ans3,severity3,bully3
38,aprilpooh15,Q: Fuck Justin bieber. He&#039;s a fucking fag...,Fuck Justin bieber. He's a fucking fag,UM excuse yuh? HE IS NOT!! Ur a fuckin idiot!...,None,Yes,1,Ur a fuckin idiot!,Yes,5,Ur a fuckin idiot,Yes,5,Ur a fuckin idiot
62,aprilpooh15,Q: Hey. Why you such a bitch?<br>A: Why thank ...,Hey. Why you such a bitch?,Why thank yuh!,None,Yes,3,"""""""why you such a bitch?""""""",Yes,5,"""""""Bitch""""""",Yes,9,Why you such a bitch?
191,aprilpooh15,Q: who decided to take a shit on your face?<br...,who decided to take a shit on your face?,im not quite sure! ill tell ya when i find ou...,None,Yes,2,shit on your face,Yes,2,who decided to take a shit on your face?,Yes,5,shit on your face
200,aprilpooh15,Q: Why do u follow the bandwagon? Formspring. ...,Why do u follow the bandwagon? Formspring. Twi...,Well im not following anything and really BAN...,None,Yes,2,Idiot! Cnt spell!,Yes,4,"""""""gay""""""",Yes,2,"""""""Gay"""""""
244,joielovesu,Q: lmao.. &quot;yurr reall funny skinny ass bi...,lmao.. yurr reall funny skinny ass bitchh.. hm...,ahhaha yuurr funnyy reall funny ndd i dntt wa...,None,Yes,8,"""""""bitchh"""" """"talkn shit"""" """"white trash skank...",Yes,10,"""""""Q: lmao.. """"yurr reall funny skinny ass bit...",Yes,10,Later white trash skank. BITCH!


**Tokenizer**
Build a custom function that can gather actual words from a give string.

In [23]:
stop_words = stopwords.words("english")  ##English words which does not add much meaning to a sentence.
lemmatizer = WordNetLemmatizer() 
enchant_dict = enchant.Dict("en_US")     ##used to check the spelling of a word, gives suggestions to correct words.

def tokenize(text):
    # normalize case and remove punctuation
    text = re.sub(r"[^a-zA-Z0-9]", " ", text.lower())
    
    # tokenize text
    tokens = word_tokenize(text)
    
    # lemmatize andremove stop words
    tokens = [lemmatizer.lemmatize(word) for word in tokens if word not in stop_words]
    
    #tokens = [word for word in tokens if enchant_dict.check(word)]

    return tokens

In [41]:
for i in range(0,339):      #range..339
    print(bully_df.ques[i])
    print(tokenize(bully_df.ques[i]),"\n")

Fuck Justin bieber. He's a fucking fag
['fuck', 'justin', 'bieber', 'fucking', 'fag'] 

Hey. Why you such a bitch?
['hey', 'bitch'] 

who decided to take a shit on your face?
['decided', 'take', 'shit', 'face'] 

Why do u follow the bandwagon? Formspring. Twitter. What's next gay Justin beiber.
['u', 'follow', 'bandwagon', 'formspring', 'twitter', 'next', 'gay', 'justin', 'beiber'] 

lmao.. yurr reall funny skinny ass bitchh.. hm.. that isn't really much of an insult now is it? what if i was fat? lol u suck at talkn shit :] later white trash skank :] ur super ugly nd that guy u like really isnt gonna come back around for u. laterrrr
['lmao', 'yurr', 'reall', 'funny', 'skinny', 'as', 'bitchh', 'hm', 'really', 'much', 'insult', 'fat', 'lol', 'u', 'suck', 'talkn', 'shit', 'later', 'white', 'trash', 'skank', 'ur', 'super', 'ugly', 'nd', 'guy', 'u', 'like', 'really', 'isnt', 'gon', 'na', 'come', 'back', 'around', 'u', 'laterrrr'] 

lol how cute u have friends that back u up :] doesnt matter

In [38]:
bully_df

,userid,post,ques,ans,asker,ans1,severity1,bully1,ans2,severity2,bully2,ans3,severity3,bully3
0,aprilpooh15,Q: Fuck Justin bieber. He&#039;s a fucking fag...,Fuck Justin bieber. He's a fucking fag,UM excuse yuh? HE IS NOT!! Ur a fuckin idiot!...,None,Yes,1,Ur a fuckin idiot!,Yes,5,Ur a fuckin idiot,Yes,5,Ur a fuckin idiot
1,aprilpooh15,Q: Hey. Why you such a bitch?<br>A: Why thank ...,Hey. Why you such a bitch?,Why thank yuh!,None,Yes,3,"""""""why you such a bitch?""""""",Yes,5,"""""""Bitch""""""",Yes,9,Why you such a bitch?
2,aprilpooh15,Q: who decided to take a shit on your face?<br...,who decided to take a shit on your face?,im not quite sure! ill tell ya when i find ou...,None,Yes,2,shit on your face,Yes,2,who decided to take a shit on your face?,Yes,5,shit on your face
3,aprilpooh15,Q: Why do u follow the bandwagon? Formspring. ...,Why do u follow the bandwagon? Formspring. Twi...,Well im not following anything and really BAN...,None,Yes,2,Idiot! Cnt spell!,Yes,4,"""""""gay""""""",Yes,2,"""""""Gay"""""""
4,joielovesu,Q: lmao.. &quot;yurr reall funny skinny ass bi...,lmao.. yurr reall funny skinny ass bitchh.. hm...,ahhaha yuurr funnyy reall funny ndd i dntt wa...,None,Yes,8,"""""""bitchh"""" """"talkn shit"""" """"white trash skank...",Yes,10,"""""""Q: lmao.. """"yurr reall funny skinny ass bit...",Yes,10,Later white trash skank. BITCH!
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
334,outlaw9000,Q: okay i added some people who want sexual s...,okay i added some people who want sexual spam...,okies,None,Yes,1,some people who want sexual spam i dont do tha...,Yes,6,sexual spam,Yes,6,sexual spam
335,outlaw9000,Q: r u a drty slut?<br>A: yep ! !,r u a drty slut?,yep ! !,None,Yes,4,r u a dirty slut,Yes,3,dirty slut,Yes,2,drty slut
336,outlaw9000,Q: When caught early the embryo isn&#039;t mu...,When caught early the embryo isn't much diffe...,I'm tired of debating this with you. I have s...,None,Yes,1,get rid of it,Yes,3,killing sperm,Yes,7,Q: When caught early the embryo isn't much dif...
337,outlaw9000,Q: You&#039;re a bushwhacking alchy piece of ...,You're a bushwhacking alchy piece of shit scrub.,and you're blocked,None,Yes,3,piece of shit,Yes,9,You're a bushwhacking alchy piece of shit scrub,Yes,6,You're a bushwhacking alchy piece of shit scrub.


In [37]:
df['ans']

0                 I like too many songs to have a favorite
1                                       </3 ? haha jk! <33
2                                 Really?!?! Thanks?! haha
3                                                       ;(
4                                                  *RAWR*?
                               ...                        
12766     Not incest  because we would not be related s...
12768     take the battery cable off   and make her sta...
12769                       thank you ! ! I AM Awesome ! !
12770         all the time. sometimes more than once a day
12771       oh O.K ! ! nice make sure it's Stolie's Please
Name: ans, Length: 11089, dtype: object